In [8]:
import os
import json
import spacy
import pandas as pd
from spacy_download import load_spacy


# Will download the model if it isn't installed yet
nlp = load_spacy("en_core_web_lg")  

In [9]:
df = pd.read_json('./News_Category_Dataset_v3.json', encoding='utf-8', lines=True)
df['cleaned_text'] = None

In [10]:
# I decided to merge similar categories to improve accuracy
df.category = df.category.map(lambda x: "WORLDPOST" if x == "THE WORLDPOST" else x)
df.category = df.category.map(lambda x: "ARTS & CULTURE" if x == "ARTS" else x)
df.category = df.category.map(lambda x: "ARTS & CULTURE" if x == "CULTURE & ARTS" else x)
df.category = df.category.map(lambda x: "STYLE & BEAUTY" if x == "STYLE" else x)

In [11]:
categories = df.groupby('category')
num_topics = categories.size()

In [12]:
# trying to improve accuracy by combining headline and short description
df['text'] = df.headline + " " + df.short_description

In [13]:
def pre_process_document(text):
    doc = nlp(text)
    filtered_tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return ' '.join(filtered_tokens)

In [14]:
# must do some nlp processing to improve the topic modeling
# it looks like the topics generated used a lot of stop words
# aded a lemmatization step for better accuracy
df['cleaned_text'] = df['text'].apply(pre_process_document)

In [ ]:
df.head()

In [ ]:
df.to_json('cleaned_kaggle_data.json', orient='records', lines=True)